# Proyecto Individual 03 de Henry por Guillermo Fernandez

#### Importar librerías necesarias

In [1]:
# Librería para trabajar con los datasets
import pandas as pd

#### Obtener la fuente de los datos

Mediante el uso de la API del sitio web https://datosabiertos.enacom.gob.ar/ se obtiene un dataset que incluye toda la información disponible del sitio, para revisar cuáles se utilizarán

In [2]:
DF_list = pd.read_json("https://api.datosabiertos.enacom.gob.ar/api/v2/datastreams.json?auth_key=1Ytx5gAWqqJhXxsRrE6Ckj8jxJVdc9BzpxQ1N5ax")

Se busca aquellos datastreams donde aparezca la palabra internet, y se imprimen los enlaces

In [3]:
# Utilizamos el campo descripcion, porque da mejor resultado que tag o title
for ind, elem in enumerate(DF_list.description):
    if "nternet" in elem:
        print(DF_list.iloc[ind].link)

http://datosabiertos.enacom.gob.ar/dataviews/240978/penetracion-del-internet-fijo-por-provincia-accesos-por-cada-100-habitantes/
http://datosabiertos.enacom.gob.ar/dataviews/240980/penetracion-del-internet-fijo-por-provincia-accesos-por-cada-100-hogares/
http://datosabiertos.enacom.gob.ar/dataviews/240981/penetracion-nacional-del-internet-fijo/
http://datosabiertos.enacom.gob.ar/dataviews/240904/acceso-a-internet-fijo-por-rangos-de-velocidad-de-bajada-y-provincia/
http://datosabiertos.enacom.gob.ar/dataviews/241420/total-nacional-de-accesos-a-internet-fijo-por-velocidad-de-bajada/
http://datosabiertos.enacom.gob.ar/dataviews/248812/accesos-a-internet-fijo-por-velocidad-bajada-y-provincia/
http://datosabiertos.enacom.gob.ar/dataviews/240898/acceso-a-internet-fijo-por-tecnologia-y-provincia/
http://datosabiertos.enacom.gob.ar/dataviews/240902/total-nacional-de-accesos-a-internet-fijo-por-tipo-de-tecnologia/
http://datosabiertos.enacom.gob.ar/dataviews/240977/ingresos-por-la-operacion-del

Luego de entrar en cada enlace, a través de la API del sitio web se obtiene el dataset

#### Cargar los datasets y explorarlos (EDA)

In [4]:
DF_operadores_reclamos = pd.read_csv("http://api.datosabiertos.enacom.gob.ar/api/v2/datastreams/OPERA-CON-EL-MAYOR-NUMER/data.csv/?auth_key=1Ytx5gAWqqJhXxsRrE6Ckj8jxJVdc9BzpxQ1N5ax", thousands = ".")
DF_operadores_reclamos.head()

,Operador,Cantidad
0,Telecom Argentina S.A.,2824
1,Telefonica de Argentina S.A.,1435
2,TELECENTRO S.A.,846
3,Otros,596


Este dataset contiene un feature que no es muy relevantes para el análisis. Se descarta

In [5]:
DF_total_accesos_por_banda = pd.read_csv("http://api.datosabiertos.enacom.gob.ar/api/v2/datastreams/TOTAL-NACIO-DE-ACCES-48866/data.csv/?auth_key=1Ytx5gAWqqJhXxsRrE6Ckj8jxJVdc9BzpxQ1N5ax", thousands = ".")
DF_total_accesos_por_banda.head()

,Año,Trimestre,Banda ancha fija,Dial up,Total,Periodo
0,2022,2,10946248,12436,10958684,Abr-Jun 2022
1,2022,1,10611390,12619,10624009,Ene-Mar 2022
2,2021,4,10476933,12861,10489794,Oct-Dic 2021
3,2021,3,10075184,10357,10085541,Jul-Sept 2021
4,2021,2,9852702,10382,9863084,Abr-Jun 2021


Este dataset es el resultado de la suma total de provincias, por lo que su información está presente en otro dataset. Se descarta

#### Accesos por banda por provincia

In [6]:
DF_accesos_por_banda_por_provincia = pd.read_csv("http://api.datosabiertos.enacom.gob.ar/api/v2/datastreams/BANDA-ANCHA-Y-BANDA-ANGOS/data.csv/?auth_key=1Ytx5gAWqqJhXxsRrE6Ckj8jxJVdc9BzpxQ1N5ax", thousands=".")
DF_accesos_por_banda_por_provincia.head()

,Año,Trimestre,Provincia,Banda ancha fija,Dial up,Total
0,2022,2,Buenos Aires,4660984,6.199,4667183
1,2022,2,Capital Federal,1534626,2.145,1536771
2,2022,2,Catamarca,67007,1,67008
3,2022,2,Chaco,143559,5,143564
4,2022,2,Chubut,161901,904,162805


Este dataset contiene features de interés para el análisis, y está segregado por provincia. Se procede a explorarlo

In [7]:
DF_accesos_por_banda_por_provincia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 816 entries, 0 to 815
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Año               816 non-null    int64 
 1   Trimestre         816 non-null    int64 
 2   Provincia         816 non-null    object
 3   Banda ancha fija  816 non-null    int64 
 4   Dial up           814 non-null    object
 5   Total             816 non-null    int64 
dtypes: int64(4), object(2)
memory usage: 38.4+ KB


Se observa que hay features cuyo dtype es incorrecto, se exploran los valores

In [8]:
DF_accesos_por_banda_por_provincia["Dial up"].value_counts()

- 0      57
1        49
3        36
2        32
13       19
         ..
2.477     1
108       1
50        1
410       1
612       1
Name: Dial up, Length: 307, dtype: int64

Se toman medidas correctivas

In [9]:
DF_accesos_por_banda_por_provincia["Dial up"].replace("\D","",regex=True,inplace=True)
DF_accesos_por_banda_por_provincia["Dial up"].fillna(0,inplace=True)
DF_accesos_por_banda_por_provincia["Dial up"] = DF_accesos_por_banda_por_provincia["Dial up"].astype(int)

Se pasa al siguiente dataset

In [10]:
DF_total_accesos_por_tecnologia = pd.read_csv("http://api.datosabiertos.enacom.gob.ar/api/v2/datastreams/TOTAL-NACIO-DE-ACCES-A/data.csv/?auth_key=1Ytx5gAWqqJhXxsRrE6Ckj8jxJVdc9BzpxQ1N5ax")
DF_total_accesos_por_tecnologia.head()

,Año,Trimestre,ADSL,Cablemodem,Fibra óptica,Wireless,Otros,Total,Periodo
0,2022,2,1.468.333,5.979.214,2.723.285,556.243,231.609,10.958.684,Abr-Jun 2022
1,2022,1,1.533.240,6.073.426,2.219.533,545.814,251.996,10.624.009,Ene-Mar 2022
2,2021,4,1.657.615,5.984.240,2.072.236,523.107,252.596,10.489.794,Oct-Dic 2021
3,2021,3,1.950.631,5.826.257,1.566.048,492.415,250.190,10.085.541,Jul-Sept 2021
4,2021,2,2.018.587,5.641.731,1.472.246,476.968,253.552,9.863.084,Abr-Jun 2021


Este dataset es el resultado de la suma total de provincias, por lo que su información está presente en otro dataset. Se descarta

#### Accesos por tecnología por provincia

In [11]:
DF_accesos_por_tecnologia_por_provincia = pd.read_csv("http://api.datosabiertos.enacom.gob.ar/api/v2/datastreams/ACCES-A-INTER-FIJO-POR/data.csv/?auth_key=1Ytx5gAWqqJhXxsRrE6Ckj8jxJVdc9BzpxQ1N5ax", thousands=".")
DF_accesos_por_tecnologia_por_provincia.head()

,Año,Trimestre,Provincia,ADSL,Cablemodem,Fibra óptica,Wireless,Otros,Total
0,2022,2,Buenos Aires,350652.0,2719613.0,1399043.0,126847.0,71028.0,4667183.0
1,2022,2,Capital Federal,145491.0,1232226.0,121506.0,5757.0,31791.0,1536771.0
2,2022,2,Catamarca,11928.0,9430.0,42087.0,1329.0,2234.0,67008.0
3,2022,2,Chaco,29552.0,61284.0,42194.0,8178.0,2356.0,143564.0
4,2022,2,Chubut,45377.0,69242.0,9574.0,29784.0,8828.0,162805.0


Este dataset contiene features de interés para el análisis, y está segregado por provincia. Se procede a explorarlo

In [12]:
DF_accesos_por_tecnologia_por_provincia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 817 entries, 0 to 816
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Año           817 non-null    object 
 1   Trimestre     817 non-null    object 
 2   Provincia     816 non-null    object 
 3   ADSL          816 non-null    float64
 4   Cablemodem    816 non-null    float64
 5   Fibra óptica  816 non-null    float64
 6   Wireless      816 non-null    float64
 7   Otros         816 non-null    float64
 8   Total         816 non-null    float64
dtypes: float64(6), object(3)
memory usage: 57.6+ KB


Se observa que hay features cuyo dtype es incorrecto, se exploran los valores

In [13]:
DF_accesos_por_tecnologia_por_provincia.Año.value_counts()

2021      96
2020      96
2018      96
2017      96
2016      96
2015      96
2014      96
2019 *    72
2022      48
2019      24
*          1
Name: Año, dtype: int64

In [14]:
DF_accesos_por_tecnologia_por_provincia.Trimestre.value_counts()

2                                                                                                                                     192
1                                                                                                                                     192
4                                                                                                                                     192
3                                                                                                                                     168
3 *                                                                                                                                    24
2 *                                                                                                                                    24
1 *                                                                                                                                    24
Los datos provinciales no coincide

Se toman medidas correctivas

In [15]:
DF_accesos_por_tecnologia_por_provincia.Año.replace("\D","",regex=True,inplace=True)
DF_accesos_por_tecnologia_por_provincia.Año.replace("",0,inplace=True)
DF_accesos_por_tecnologia_por_provincia.Año = DF_accesos_por_tecnologia_por_provincia.Año.astype(int)

DF_accesos_por_tecnologia_por_provincia.Trimestre.replace("\D","",regex=True,inplace=True)
DF_accesos_por_tecnologia_por_provincia.Trimestre.replace("",0,inplace=True)
DF_accesos_por_tecnologia_por_provincia.Trimestre = DF_accesos_por_tecnologia_por_provincia.Trimestre.astype(int)

DF_accesos_por_tecnologia_por_provincia.fillna(0,inplace=True)
DF_accesos_por_tecnologia_por_provincia.ADSL = DF_accesos_por_tecnologia_por_provincia.ADSL.astype(int)
DF_accesos_por_tecnologia_por_provincia.Cablemodem = DF_accesos_por_tecnologia_por_provincia.Cablemodem.astype(int)
DF_accesos_por_tecnologia_por_provincia['Fibra óptica'] = DF_accesos_por_tecnologia_por_provincia['Fibra óptica'].astype(int)
DF_accesos_por_tecnologia_por_provincia.Wireless = DF_accesos_por_tecnologia_por_provincia.Wireless.astype(int)
DF_accesos_por_tecnologia_por_provincia.Otros = DF_accesos_por_tecnologia_por_provincia.Otros.astype(int)
DF_accesos_por_tecnologia_por_provincia.Total = DF_accesos_por_tecnologia_por_provincia.Total.astype(int)

Se pasa al siguiente dataset

In [16]:
DF_accesos_por_tecnologia_por_localidad = pd.read_csv("http://api.datosabiertos.enacom.gob.ar/api/v2/datastreams/ACCES-A-INTER-FIJO-62463/data.csv/?auth_key=1Ytx5gAWqqJhXxsRrE6Ckj8jxJVdc9BzpxQ1N5ax")
DF_accesos_por_tecnologia_por_localidad.head()

,Provincia,Partido,Localidad,Link Indec,ADSL,CABLEMODEM,DIAL UP,FIBRA OPTICA,OTROS,SATELITAL,WIMAX,WIRELESS,Total general,Unnamed: 13
0,BUENOS AIRES,25 de Mayo,25 de Mayo,6854100,873,4.704,- 0,2,- 0,975,- 0,664,7.218,NaN
1,BUENOS AIRES,25 de Mayo,Del Valle,6854020,181,- 0,- 0,- 0,10,1,- 0,- 0,192,NaN
2,BUENOS AIRES,25 de Mayo,Gobernador Ugarte,6854040,- 0,- 0,- 0,- 0,- 0,- 0,- 0,181,181,NaN
3,BUENOS AIRES,25 de Mayo,Norberto de la Riestra,6854060,- 0,782,- 0,6,167,- 0,- 0,327,1.282,NaN
4,BUENOS AIRES,25 de Mayo,Lucas Monteverde,6854050,- 0,- 0,- 0,- 0,- 0,- 0,- 0,6,6,NaN


Este dataset está segregado por localidad. Al considerar las provincias, se puede descartar

In [17]:
DF_total_accesos_por_velocidad = pd.read_csv("http://api.datosabiertos.enacom.gob.ar/api/v2/datastreams/TOTAL-NACIO-DE-ACCES-32663/data.csv/?auth_key=1Ytx5gAWqqJhXxsRrE6Ckj8jxJVdc9BzpxQ1N5ax")
DF_total_accesos_por_velocidad.head()

,Año,Trimestre,Hasta 512 kbps,Entre 512 Kbps y 1 Mbps,Entre 1 Mbps y 6 Mbps,Entre 6 Mbps y 10 Mbps,Entre 10 Mbps y 20 Mbps,Entre 20 Mbps y 30 Mbps,Más de 30 Mbps,OTROS,Total,Unnamed: 11
0,2022,2,33.667,99.498,1.193.090,1.197.030,856.562,485.321,6.741.922,351.594,10.958.684,NaN
1,2022,1,34.890,104.840,1.263.273,1.209.148,967.508,509.830,6.336.187,198.333,10.624.009,NaN
2,2021,4,41.262,28.521,1.413.208,1.245.333,976.539,558.358,6.032.322,194.251,10.489.794,NaN
3,2021,3,40.174,41.437,2.550.229,1.095.772,710.122,536.364,4.948.174,163.269,10.085.541,NaN
4,2021,2,40.172,42.024,2.531.271,1.080.279,693.277,647.401,4.661.291,167.369,9.863.084,NaN


Este dataset es el resultado de la suma total de provincias, por lo que su información está presente en otro dataset. Se descarta

In [18]:
DF_accesos_por_velocidad_por_provincia = pd.read_csv("http://api.datosabiertos.enacom.gob.ar/api/v2/datastreams/ACCES-A-INTER-FIJO-97624/data.csv/?auth_key=1Ytx5gAWqqJhXxsRrE6Ckj8jxJVdc9BzpxQ1N5ax", decimal=",")
DF_accesos_por_velocidad_por_provincia.head()

,Año,Trimestre,Provincia,Otros,"0,256 Mbps","0,375 Mbps","0,625 Mbps","0,5 Mbps","0,512 Mbps","0,75 Mbps",...,70 Mbps,71 Mbps,75 Mbps,77 Mbps,78 Mbps,80 Mbps,81 Mbps,83 Mbps,82 Mbps,86 Mbps
0,2022,2,Buenos Aires,"65,849",19,- 0,- 0,30.428,192,"4,611",...,"3,163",- 0,"104,021",- 0,2,216,- 0,1,- 0,- 0
1,2022,2,Capital Federal,"105,607",31,- 0,- 0,451.000,35,"1,307",...,3,- 0,"32,185",- 0,- 0,8,- 0,- 0,- 0,- 0
2,2022,2,Catamarca,"2,207",- 0,- 0,- 0,34.000,37,- 0,...,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0
3,2022,2,Chaco,"3,708",- 0,225,111,234.000,2,- 0,...,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0
4,2022,2,Chubut,"20,021",8,- 0,- 0,69.000,32,836,...,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0


Este dataset, si bien está segregado por provincia, posee una gran cantidad de columnas, la mayoría de ellas con ceros. Se utilizará la versión que viene por rangos de velocidad, por lo que se descarta

In [19]:
DF_accesos_por_velocidad_por_localidad = pd.read_csv("http://api.datosabiertos.enacom.gob.ar/api/v2/datastreams/ACCES-A-INTER-FIJO-16249/data.csv/?auth_key=1Ytx5gAWqqJhXxsRrE6Ckj8jxJVdc9BzpxQ1N5ax")
DF_accesos_por_velocidad_por_localidad.head()

,Provincia,Partido,Localidad,Link Indec,Otros,"0,256 Mbps","0,375 Mbps","0,5 Mbps","0,512 Mbps","0,625 Mbps",...,75 Mbps,78 Mbps,80 Mbps,82 Mbps,83 Mbps,85 Mbps,90 Mbps,92 Mbps,95 Mbps,100 Mbps
0,BUENOS AIRES,25 de Mayo,25 de Mayo,6854100,NaN,NaN,NaN,2.0,NaN,NaN,...,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BUENOS AIRES,25 de Mayo,Agustín Mosconi,6854010,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BUENOS AIRES,25 de Mayo,Del Valle,6854020,NaN,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BUENOS AIRES,25 de Mayo,Ernestina,6854030,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,BUENOS AIRES,25 de Mayo,Gobernador Ugarte,6854040,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Este dataset también posee gran cantidad de columnas, y segregación por localidad. Se descarta

#### Accesos por rangos de velocidad por provincia

In [20]:
DF_accesos_por_rangos_velocidad_por_provincia = pd.read_csv("http://api.datosabiertos.enacom.gob.ar/api/v2/datastreams/ACCES-A-INTER-FIJO-23248/data.csv/?auth_key=1Ytx5gAWqqJhXxsRrE6Ckj8jxJVdc9BzpxQ1N5ax", thousands=".")
DF_accesos_por_rangos_velocidad_por_provincia.head()

,Año,Trimestre,Provincia,HASTA 512 kbps,+ 512 Kbps - 1 Mbps,+ 1 Mbps - 6 Mbps,+ 6 Mbps - 10 Mbps,+ 10 Mbps - 20 Mbps,+ 20 Mbps - 30 Mbps,+ 30 Mbps,OTROS,Total
0,2022,2,Buenos Aires,30639,28323,295238,307554,273954,129869,3535757,65849.0,4667183
1,2022,2,Capital Federal,517,5980,36036,72434,55372,31571,1229254,105607.0,1536771
2,2022,2,Catamarca,71,413,3678,6155,7165,3393,43926,2207.0,67008
3,2022,2,Chaco,461,1047,17603,20109,8391,15974,76271,3708.0,143564
4,2022,2,Chubut,109,1444,45804,31217,34290,14526,15394,20021.0,162805


Este dataset contiene features de interés para el análisis, y está segregado por provincia. Se procede a explorarlo

In [21]:
DF_accesos_por_rangos_velocidad_por_provincia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 816 entries, 0 to 815
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Año                  816 non-null    int64  
 1   Trimestre            816 non-null    int64  
 2   Provincia            816 non-null    object 
 3   HASTA 512 kbps       816 non-null    int64  
 4   + 512 Kbps - 1 Mbps  816 non-null    int64  
 5   + 1 Mbps - 6 Mbps    816 non-null    int64  
 6   + 6 Mbps - 10 Mbps   816 non-null    int64  
 7   + 10 Mbps - 20 Mbps  816 non-null    int64  
 8   + 20 Mbps - 30 Mbps  816 non-null    int64  
 9   + 30 Mbps            816 non-null    int64  
 10  OTROS                810 non-null    float64
 11  Total                816 non-null    int64  
dtypes: float64(1), int64(10), object(1)
memory usage: 76.6+ KB


Se observan dtypes incorrectos. Se aplican correcciones

In [22]:
DF_accesos_por_rangos_velocidad_por_provincia.fillna(0,inplace=True)
DF_accesos_por_rangos_velocidad_por_provincia.OTROS = DF_accesos_por_rangos_velocidad_por_provincia.OTROS.astype(int)

Se avanza al siguiente dataset

In [23]:
DF_velocidad_media_nacional = pd.read_csv("http://api.datosabiertos.enacom.gob.ar/api/v2/datastreams/VELOC-PROME-DE-BAJAD-51733/data.csv/?auth_key=1Ytx5gAWqqJhXxsRrE6Ckj8jxJVdc9BzpxQ1N5ax", decimal = ",")
DF_velocidad_media_nacional.head()

,Año,Trimestre,Mbps (Media de bajada),Trimestre.1
0,2022,2,58.44,Abr-Jun 2022
1,2022,1,55.11,Ene-Mar 2022
2,2021,4,52.34,Oct-Dic 2021
3,2021,3,48.46,Jul-Sept 2021
4,2021,2,45.63,Abr-Jun 2021


Este dataset es el resultado de la suma total de provincias, por lo que su información está presente en otro dataset. Se descarta

#### Velocidad media por provincia

In [24]:
DF_velocidad_media_por_provincia = pd.read_csv("http://api.datosabiertos.enacom.gob.ar/api/v2/datastreams/VELOC-PROME-DE-BAJAD-DE/data.csv/?auth_key=1Ytx5gAWqqJhXxsRrE6Ckj8jxJVdc9BzpxQ1N5ax")
DF_velocidad_media_por_provincia.head()

,Año,Trimestre,Provincia,Mbps (Media de bajada)
0,2022,2,Buenos Aires,66
1,2022,2,Capital Federal,94
2,2022,2,Catamarca,57
3,2022,2,Chaco,51
4,2022,2,Chubut,15


Este dataset contiene features de interés.

In [25]:
DF_velocidad_media_por_provincia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 816 entries, 0 to 815
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Año                     816 non-null    int64 
 1   Trimestre               816 non-null    int64 
 2   Provincia               816 non-null    object
 3   Mbps (Media de bajada)  816 non-null    int64 
dtypes: int64(3), object(1)
memory usage: 25.6+ KB


#### Ingresos por servicio

In [26]:
DF_ingresos_servicio = pd.read_csv("http://api.datosabiertos.enacom.gob.ar/api/v2/datastreams/INGRE-POR-LA-OPERA-DEL/data.csv/?auth_key=1Ytx5gAWqqJhXxsRrE6Ckj8jxJVdc9BzpxQ1N5ax", thousands=".")
DF_ingresos_servicio.head()

,Año,Trimestre,Ingresos (miles de pesos),Periodo
0,2022,2,60942843,Abr-Jun 2022
1,2022,1,56195811,Ene-Mar 2022
2,2021,4,45467887,Oct-Dic 2021
3,2021,3,42999944,Jul-Sept 2021
4,2021,2,38239667,Abr-Jun 2021


Este dataset contiene features de interés.

In [27]:
DF_ingresos_servicio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34 entries, 0 to 33
Data columns (total 4 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Año                        34 non-null     int64 
 1   Trimestre                  34 non-null     int64 
 2   Ingresos (miles de pesos)  34 non-null     int64 
 3   Periodo                    34 non-null     object
dtypes: int64(3), object(1)
memory usage: 1.2+ KB


#### Penetración del servicio

In [28]:
DF_penetracion_por_provincia_100habitantes = pd.read_csv("https://api.datosabiertos.enacom.gob.ar/api/v2/datastreams/PENET-DEL-INTER-FIJO-POR/data.csv/?auth_key=1Ytx5gAWqqJhXxsRrE6Ckj8jxJVdc9BzpxQ1N5ax")
DF_penetracion_por_provincia_100habitantes.head()

,Año,Trimestre,Provincia,Accesos por cada 100 hab
0,2022,2,Buenos Aires,26
1,2022,2,Capital Federal,50
2,2022,2,Catamarca,16
3,2022,2,Chaco,12
4,2022,2,Chubut,26


Este dataset contiene feature de interés

In [29]:
DF_penetracion_por_provincia_100habitantes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 816 entries, 0 to 815
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Año                       816 non-null    int64 
 1   Trimestre                 816 non-null    int64 
 2   Provincia                 816 non-null    object
 3   Accesos por cada 100 hab  816 non-null    int64 
dtypes: int64(3), object(1)
memory usage: 25.6+ KB


In [30]:
DF_penetracion_por_provincia_100hogares = pd.read_csv("http://api.datosabiertos.enacom.gob.ar/api/v2/datastreams/PENET-DEL-INTER-FIJO-51614/data.csv/?auth_key=1Ytx5gAWqqJhXxsRrE6Ckj8jxJVdc9BzpxQ1N5ax", decimal=",")
DF_penetracion_por_provincia_100hogares.head()

,Año,Trimestre,Provincia,Accesos por cada 100 hogares
0,2022,2,Buenos Aires,77.58
1,2022,2,Capital Federal,121.31
2,2022,2,Catamarca,62.40
3,2022,2,Chaco,43.79
4,2022,2,Chubut,83.19


Este dataset es muy similar al anterior, también tiene features de interés

In [31]:
DF_penetracion_por_provincia_100hogares.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 816 entries, 0 to 815
Data columns (total 4 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Año                           816 non-null    int64  
 1   Trimestre                     816 non-null    int64  
 2   Provincia                     816 non-null    object 
 3   Accesos por cada 100 hogares  816 non-null    float64
dtypes: float64(1), int64(2), object(1)
memory usage: 25.6+ KB


Se avanza al próximo dataset

In [32]:
DF_penetracion_nacional = pd.read_csv("http://api.datosabiertos.enacom.gob.ar/api/v2/datastreams/PENET-NACIO-DEL-INTER-FIJO/data.csv/?auth_key=1Ytx5gAWqqJhXxsRrE6Ckj8jxJVdc9BzpxQ1N5ax", decimal=",")
DF_penetracion_nacional.head()

,Año,Trimestre,Accesos por cada 100 hogares,Accesos por cada 100 hab,Periodo
0,2022,2,75.97,23.72,Abr-Jun 2022
1,2022,1,73.88,23.05,Ene-Mar 2022
2,2021,4,73.18,22.81,Oct-Dic 2021
3,2021,3,70.58,21.98,Jul-Sept 2021
4,2021,2,69.24,21.55,Abr-Jun 2021


Este dataset es el resultado de la suma total de provincias, por lo que su información está presente en otro dataset. Se descarta

In [33]:
DF_conectividad = pd.read_csv("http://api.datosabiertos.enacom.gob.ar/api/v2/datastreams/CONEC-AL-SERVI-DE-INTER/data.csv/?auth_key=1Ytx5gAWqqJhXxsRrE6Ckj8jxJVdc9BzpxQ1N5ax")
DF_conectividad.head()

,Provincia,Partido,Localidad,Poblacion,ADSL,CABLEMODEM,DIALUP,FIBRAOPTICA,SATELITAL,WIRELESS,TELEFONIAFIJA,3G,4G,link,Latitud,Longitud
0,BUENOS AIRES,25 de Mayo,25 de Mayo,23408,SI,SI,--,SI,SI,SI,SI,SI,SI,6854100.0,-35.433939,-60.173121
1,BUENOS AIRES,25 de Mayo,Agustín Mosconi,280,--,--,--,--,--,--,SI,--,--,6854010.0,-35.739217,-60.559063
2,BUENOS AIRES,25 de Mayo,Del Valle,899,SI,--,--,--,SI,--,--,SI,SI,6854020.0,-35.897345,-60.731609
3,BUENOS AIRES,25 de Mayo,Ernestina,145,SI,--,--,--,--,--,--,--,--,6854030.0,-35.270234,-59.557503
4,BUENOS AIRES,25 de Mayo,Gobernador Ugarte,547,--,--,--,--,--,SI,--,--,--,6854040.0,-35.164397,-60.081358


Este dataset está segregado por localidad, y su información también está disponible en otro dataset, en forma cuantitativa en vez de categórica. Se descarta

In [34]:
DF_conectividad_listado = pd.read_csv("http://api.datosabiertos.enacom.gob.ar/api/v2/datastreams/LISTA-DE-LOCAL-CON-CONEC/data.csv/?auth_key=1Ytx5gAWqqJhXxsRrE6Ckj8jxJVdc9BzpxQ1N5ax")
DF_conectividad_listado.head()

,Provincia,Partido,Localidad,ADSL,CABLEMODEM,DIALUP,FIBRAOPTICA,4G,3G,TELEFONIAFIJA,WIRELESS,SATELITAL
0,BUENOS AIRES,25 de Mayo,25 de Mayo,SI,SI,--,SI,SI,SI,SI,SI,SI
1,BUENOS AIRES,25 de Mayo,Agustín Mosconi,--,--,--,--,--,--,SI,--,--
2,BUENOS AIRES,25 de Mayo,Del Valle,SI,--,--,--,SI,SI,--,--,SI
3,BUENOS AIRES,25 de Mayo,Ernestina,SI,--,--,--,--,--,--,--,--
4,BUENOS AIRES,25 de Mayo,Gobernador Ugarte,--,--,--,--,--,--,--,SI,--


Similar al anterior. No tiene relevancia

### Conclusiones

+ Se cargaron en total 18 datasets
+ Datasets repetidos, segregados por localidad, provincias y total nacional
+ Datasets que no aportan datos relevantes de análisis
+ Al unir los datasets relevantes, podemos obtener uno solo con toda la información necesaria para PowerBI

Se unen los datasets

In [39]:
DF = DF_accesos_por_banda_por_provincia.merge(DF_accesos_por_tecnologia_por_provincia).merge(DF_accesos_por_rangos_velocidad_por_provincia)\
     .merge(DF_velocidad_media_por_provincia).merge(DF_ingresos_servicio).merge(DF_penetracion_por_provincia_100habitantes)\
     .merge(DF_penetracion_por_provincia_100hogares)
DF.sample(3)

,Año,Trimestre,Provincia,Banda ancha fija,Dial up,Total,ADSL,Cablemodem,Fibra óptica,Wireless,...,+ 6 Mbps - 10 Mbps,+ 10 Mbps - 20 Mbps,+ 20 Mbps - 30 Mbps,+ 30 Mbps,OTROS,Mbps (Media de bajada),Ingresos (miles de pesos),Periodo,Accesos por cada 100 hab,Accesos por cada 100 hogares
163,2020,4,Santa Cruz,40477,197,40674,12347,19657,1041,4659,...,4382,6634,1033,647,758,7,33539703,Oct-Dic 2020,11,37.00
164,2020,4,Santa Fe,820676,137,820813,287271,414746,62692,38712,...,203793,94246,65145,226246,23602,27,33539703,Oct-Dic 2020,23,73.58
95,2021,3,Tucumán,232447,25,232472,104489,53868,70964,2408,...,50293,21986,5300,115154,164,42,42999944,Jul-Sept 2021,14,54.64


Se revisa que todas las features sean de interés

In [42]:
DF.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 295 entries, 0 to 294
Data columns (total 22 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Año                           295 non-null    int64  
 1   Trimestre                     295 non-null    int64  
 2   Provincia                     295 non-null    object 
 3   Banda ancha fija              295 non-null    int64  
 4   Dial up                       295 non-null    int32  
 5   ADSL                          295 non-null    int32  
 6   Cablemodem                    295 non-null    int32  
 7   Fibra óptica                  295 non-null    int32  
 8   Wireless                      295 non-null    int32  
 9   Otros                         295 non-null    int32  
 10  HASTA 512 kbps                295 non-null    int64  
 11  + 512 Kbps - 1 Mbps           295 non-null    int64  
 12  + 1 Mbps - 6 Mbps             295 non-null    int64  
 13  + 6 M

Se eliminan 2 features que no son relevantes

In [41]:
DF.drop(columns=['Total','Periodo'], inplace=True)

Se exporta el csv limpio para cargar en PowerBI

In [43]:
DF.to_csv("Conexiones_a_internet_en_Argentina.csv", index=False)